## [作業重點]
了解如何使用 Sklearn 中的 hyper-parameter search 找出最佳的超參數

### 作業
請使用不同的資料集，並使用 hyper-parameter search 的方式，看能不能找出最佳的超參數組合

In [1]:
from sklearn import datasets, metrics
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.warn = ignore_warn #ignore annoying warning (from sklearn and seaborn)

In [2]:
# 讀取資料集
diabetes = datasets.load_diabetes()
diabetes.data.shape

(442, 10)

In [3]:
# 切分訓練集/測試集
X_train, X_test, y_train, y_test = train_test_split(diabetes.data, diabetes.target, test_size=0.1, random_state=42)


In [4]:
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.model_selection import KFold, GridSearchCV

def findBestGradientBoosting():
    # Choose some parameter combinations to try
    param_grid = {'n_estimators': [80 , 100, 200, 300, 400, 500], 
                  'max_depth': [2, 3, 5], 
                  'learning_rate': [0.005, 0.01,0.05,0.07, 0.08]
                 }

    # 建立模型
    clf = GradientBoostingRegressor(random_state=7)
    # Run the grid search
    grid_obj = GridSearchCV(estimator = clf,param_grid = param_grid, verbose=1)
    grid_obj = grid_obj.fit(X_train, y_train)

    # 印出最佳結果與最佳參數
    print("Best Accuracy: %f using %s" % (grid_obj.best_score_, grid_obj.best_params_))
    return grid_obj.best_estimator_

In [5]:
# Fit the best algorithm to the data. 
clf = findBestGradientBoosting()
clf.fit(X_train, y_train)

predictions = clf.predict(X_test)
print(mean_squared_error(y_test, predictions))

Fitting 3 folds for each of 90 candidates, totalling 270 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Best Accuracy: 0.439773 using {'learning_rate': 0.05, 'max_depth': 2, 'n_estimators': 100}
2576.170487333019


[Parallel(n_jobs=1)]: Done 270 out of 270 | elapsed:   23.0s finished


In [6]:
from sklearn.model_selection import KFold
import numpy as np
def run_kfold(clf):
    kf = KFold(n_splits=10)
    outcomes = []
    fold = 0
    for train_index, test_index in kf.split(diabetes.data):
        fold += 1
        X_train, X_test = diabetes.data[train_index], diabetes.data[test_index]
        y_train, y_test = diabetes.target[train_index], diabetes.target[test_index]
        clf.fit(X_train, y_train)
        predictions = clf.predict(X_test)
        accuracy = mean_squared_error(y_test, predictions)
        outcomes.append(accuracy)
        print("Fold {0} MSE: {1}".format(fold, accuracy))     
    mean_outcome = np.mean(outcomes)
    print("Mean MSE: {0}".format(mean_outcome)) 

run_kfold(clf)

Fold 1 MSE: 3003.847495722414
Fold 2 MSE: 2843.4525994325427
Fold 3 MSE: 3121.2051978314103
Fold 4 MSE: 3089.0134644335926
Fold 5 MSE: 3202.848514933675
Fold 6 MSE: 3350.42994842942
Fold 7 MSE: 3720.8663159375046
Fold 8 MSE: 2737.3783645960057
Fold 9 MSE: 4593.443031213643
Fold 10 MSE: 2171.6417544611463
Mean MSE: 3183.4126686991353
